In [ ]:
!pip install transformers tensorflow

In [ ]:
from transformers import T5Tokenizer


tokenizer = T5Tokenizer.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained('t5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import json
with open('train.json', 'r') as f:
    data = json.load(f)


documents = [item['dialogue'] for item in data]
summaries = [item['summary'] for item in data]

In [ ]:
import tensorflow as tf

def preprocess_documents(documents, tokenizer, max_length=512):

    input_ids = []
    attention_masks = []


    for doc in documents:

        inputs = tokenizer("summarize: " + doc,
                           max_length=max_length,
                           truncation=True,
                           padding='max_length',
                           return_tensors="tf")


        input_ids.append(inputs['input_ids'])
        attention_masks.append(inputs['attention_mask'])


    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)

    return input_ids, attention_masks


In [ ]:
input_ids, attention_masks = preprocess_documents(documents, tokenizer)

In [ ]:
def tokenize_summaries(summaries, tokenizer, max_length=512):
    token_ids = []

    for summary in summaries:

        encoded = tokenizer.encode_plus(
            summary,

            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_attention_mask=False,
            return_tensors='tf'
        )
        token_ids.append(encoded['input_ids'])

    return tf.concat(token_ids, axis=0)

In [ ]:
target_ids = tokenize_summaries(summaries, tokenizer)

In [ ]:
num_samples = input_ids.shape[0]
num_samples

14732

In [ ]:

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5,
    weight_decay=0.01,
    epsilon=1e-6,
    global_clipnorm=1.0,)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')


In [ ]:
from transformers import TFT5ForConditionalGeneration

In [ ]:
model = TFT5ForConditionalGeneration.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
def train_step(input_ids, attention_masks, target_ids):
    with tf.GradientTape() as tape:
        outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=target_ids)
        loss = tf.reduce_mean(outputs.loss)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

In [ ]:

num_epochs = 3
batch_size = 1

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for i in range(0, len(input_ids), batch_size):
        batch_input_ids = input_ids[i:i+batch_size]
        batch_attention_masks = attention_masks[i:i+batch_size]
        batch_target_ids = target_ids[i:i+batch_size]

        loss = train_step(batch_input_ids, batch_attention_masks, batch_target_ids)

        if (i // batch_size) % 10 == 0:
            print(f"Batch {i // batch_size}, Loss: {loss.numpy():.4f}")

    print(f"Epoch {epoch + 1} Loss: {loss.numpy():.4f}")

In [ ]:

num_epochs = 3
batch_size = 8
num_batches = 100
batch_counter = 0

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    epoch_loss = 0.0
    num_batches_in_epoch = 0
    batch_counter = 0

    for i in range(0, len(input_ids), batch_size):
        if batch_counter >= num_batches:
            break

        batch_input_ids = input_ids[i:i+batch_size]
        batch_attention_masks = attention_masks[i:i+batch_size]
        batch_target_ids = target_ids[i:i+batch_size]

        loss = train_step(batch_input_ids, batch_attention_masks, batch_target_ids)
        epoch_loss += loss.numpy()
        batch_counter += 1
        num_batches_in_epoch += 1

        if (i // batch_size) % 10 == 0:
            print(f"Batch {batch_counter}, Loss: {loss.numpy():.4f}")

    # epoch_loss /= num_batches_in_epoch
    # print(f"Epoch {epoch + 1} Loss: {epoch_loss:.4f}")



In [ ]:
def generate_summary(document, tokenizer, model, max_length=150, min_lenght=10):
    input_ids = tokenizer.encode("summarize: " + document, return_tensors="tf", max_length=max_length, truncation=True)
    output_ids = model.generate(input_ids=input_ids, max_length=max_length, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
document_to_summarize = "india is a democratic coountry. it has a great diversity. it's a land of god and godess. its one of the fastest growing economy"
generated_summary = generate_summary(document_to_summarize, tokenizer, model)
print("Generated Summary:", generated_summary)